___
___
# **Proyecto Final Data Science**
## **Pipeline de Sistema de Recomendación (MVP)**

Equipo: 2 - Quantum Insights
Integrantes:
- Felipe Varela - Product Owner
- Freddy Yaquive - Data Scientist
- Ivan Martinez - Data Scientist
- Sebastian Moya - Data Scientist
- Nicolás Lazarte - Scrum Master

Cohorte: DSFT01
___
___

Este proyecto implementa un sistema de recomendación basado en contenido. 

Utiliza Procesamiento de Lenguaje Natural (NLP) para analizar las descripciones de los productos y sugerir artículos similares.

- `products.csv`: Es la base de datos de productos (Materia prima).
- `pipeline_entrenamiento.py`: Es el código que aprende y crea el modelo
- `modelo_recomendacion_v1.pkl`: Se genera automáticamente, es el cerebro del modelo.

___

In [23]:
# Librerías
import pandas as pd
import pickle # para guardar archivos binarios (el cerebro guardado)
from sklearn.feature_extraction.text import TfidfVectorizer # para convertir texto en vectores númericos
from sklearn.metrics.pairwise import linear_kernel # para medir la similitud entre productos


___
### **1. Carga y Preparación de Datos**

___
Preparación de Datos (ETL)

In [24]:
# Definimos una función para limpiar y preparar
def cargar_y_preparar_datos(ruta_archivo):
    try: # Cargar CSV
        df = pd.read_csv(ruta_archivo)
    except FileNotFoundError:
        return None, "Archivo no encontrado"

    # Rellenar vacíos con texto vacío (para que no falle la suma de textos)
    columnas_clave = ['ProductName', 'Brand', 'Category', 'SubCategory']
    for col in columnas_clave:
        df[col] = df[col].fillna('') # Si sumamos "el producto" + NaN (nulo), Python daría error. Por eso reemplazamos por ""

    # Creamos un perfil de texto
    # Unimos todo en una sola columna
    def unir_columnas(fila):
        return f"{fila['Brand']} {fila['Category']} {fila['SubCategory']} {fila['ProductName']}"
            # Al mezclar Marca + Categoría + Nombre, creamos una "sopa de palabras" rica en contexto.

    df['perfil_texto'] = df.apply(unir_columnas, axis=1)
    
    # Limpieza minúsculas y quitar espacios
    df['perfil_texto'] = df['perfil_texto'].str.lower().str.strip()     # .strip() elimina espacios en blanco al inicio o final que puedan ensuciar el dato.
    print("Datos listos. 'perfil_texto'.")
    print("---"*10)
    return df

# Prueba de la función...
df_productos = cargar_y_preparar_datos("../databases/products.csv")

# Visualizamos cómo quedó la "sopa de palabras" de los primeros productos
print(f"\nPerfil creado: \n{df_productos['perfil_texto'].iloc[0:5]}") 

Datos listos. 'perfil_texto'.
------------------------------

Perfil creado: 
0    fresho fruits & vegetables potato, onion & tom...
1    fresho fruits & vegetables potato, onion & tom...
2    fresho eggs, meat & fish farm eggs farm eggs -...
3    fresho fruits & vegetables potato, onion & tom...
4    fresho fruits & vegetables potato, onion & tom...
Name: perfil_texto, dtype: object


___
### **2. Entrenamiento del Pipeline**

In [25]:
# Función para entrenar el pipeline
def entrenar_pipeline(df):
    # Vectorización de texto
        # TF-IDF da peso alto a palabras únicas (ej: "iPhone") y bajo a palabras comunes (ej: "celular").
        # stop_words='english' ayuda a limpiar ruido gramatical, elimina palabras que no aportan valor (el, la, the, and)
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Transformamos el texto a matriz de números
    # Cada fila es un producto, cada columna es una palabra del vocabulario total.
    matriz_tfidf = tfidf.fit_transform(df['perfil_texto'])
    print(f"Matriz creada: {matriz_tfidf.shape} (Productos x Palabras)")
    print("---"*10)

    # Cálculo de similitud. Calculamos la distancia de todos contra todos
    # 'linear_kernel' es equivalente a la Similitud del Coseno.
    matriz_similitud = linear_kernel(matriz_tfidf, matriz_tfidf)
        # Calcula el ángulo entre los vectores: si el ángulo es 0, son idénticos (similitud 1).
    print("---"*10)

    # Guardamos la matriz en un archivo .pkl (pickle)
    # Esto evita tener que recalcular la matriz cada vez que alguien usa la app.
    nombre_archivo = 'modelo_recomendacion_v1.pkl'
    with open(nombre_archivo, 'wb') as f:
        pickle.dump(matriz_similitud, f)
    print(f"Modelo guardado en '{nombre_archivo}'")
    return matriz_similitud

# Entrenamos el pipeline y guardamos el resultado en memoria
matriz_entrenada = entrenar_pipeline(df_productos)

Matriz creada: (2000, 2636) (Productos x Palabras)
------------------------------
------------------------------
Modelo guardado en 'modelo_recomendacion_v1.pkl'


___
### **3. Uso del Modelo (Lógica de Recomendación)**

In [26]:
# Función para obtener recomendaciones
def obtener_recomendaciones(nombre_producto, df, matriz_similitud):
    print(f"\nRecomendaciones para: '{nombre_producto}'...")
    
    # Crear índice para buscar rápido: Mapeamos 'Nombre del Producto' -> 'Número de Fila'
    indices = pd.Series(df.index, index=df['ProductName']).drop_duplicates()

    # Validar que el producto exista
    if nombre_producto not in indices:
        return "El producto no existe, intente con otro producto."

    # Obtener el ID numérico (índice de la matriz) del producto seleccionado
    idx = indices[nombre_producto]

    # Obtener los puntajes de similitud de ese producto contra TODOS los demás
    # enumerate crea una lista de pares: (ID_Producto, Puntaje_Similitud)
    puntajes = list(enumerate(matriz_similitud[idx]))

    # Ordenar de mayor a menor similitud (los más parecidos primero)    
    puntajes = sorted(puntajes, key=lambda x: x[1], reverse=True)

    # Tomar los 3 mejores (saltamos el [0] porque es el mismo producto)
    top_3 = puntajes[1:4]
    

    # Recuperar la información legible del DataFrame original usando los índices encontrados
    indices_recomendados = [i[0] for i in top_3]
    resultado = df[["product_id",'ProductName', 'Brand', 'Category']].iloc[indices_recomendados]
    
    return resultado

___
#### **4. Ejecución y Prueba de pipeline**

In [27]:
# Guardamos solo la columna de nombres en un archivo CSV nuevo, para tener la lista de productos completa
# df_productos['ProductName'].to_csv('../databases/lista_productos_completa.csv', index=False)

In [28]:
# Muestra 20 nombres de productos al azar para realizar la prueba
print("Productos disponibles (aleatorio)")
muestra = df_productos['ProductName'].sample(20).values
for producto in muestra:
    print(f"- {producto}")

Productos disponibles (aleatorio)
- Blush & Glow Gel Face Wash - With Peach Extracts, Removes Excess Oil
- Pet Plastic Water Bottle - Blue, Aster, L3022X6 BL
- Red Chilli Flakes
- Organic - Brown Sugar/Sakkare
- Sunscreen SPF 35 Lotion - Uva & Uvb Protection
- Organic - Almond/Badam
- Elegance Deodorant - For Men 150 ml + Gold Deodorant - For Men 150 ml
- Chicken Seekh Kabab Nawabi
- California Almonds
- Body Wash Fig Oil & White Orchid Extracts + Invigorating Handwash Refill (750ml)
- Nutrition Energy Bar - Almonds & Raisins
- L'Amante Aqua Eau De Parfum - Perfume For Men
- Sheamoist Shea Butter & Real Strawberry 24 Hrs Moisturiser
- Lube - Cheeky Cherry
- Basmati Rice/Basmati Akki - Super Premium
- Natural Soy Drink (Tetra Pak)
- Health Shampoo - Henna & Amla, For Problem Free Hair, 7 Satt Poshan
- Basmati Rice/Basmati Akki - Super
- G1 Cologne Spray - No Gas Perfume, Floral & Sweet, Skin Friendly, For Women
- 100% Natural Premium Californian Almonds


In [29]:
# Buscador
palabra_clave = "Chia Seeds"  # <--- Cambia esto por lo que busques (ej: 'Rice', 'Oil')

# Filtramos los productos que contengan esa palabra (sin importar mayúsculas/minúsculas)
resultados = df_productos[df_productos['ProductName'].str.contains(palabra_clave, case=False)]['ProductName'].unique()

print(f"Productos encontrados con '{palabra_clave}': {len(resultados)}")
print("---"*10)
for p in resultados[:10]: # Solo mostramos los primeros 10 para no saturar la pantalla
    print(f"- {p}")

Productos encontrados con 'Chia Seeds': 3
------------------------------
- Black Chia Seeds
- Natives Chia Seeds
- Chia Seeds


In [40]:
# Ejemplo: si encontraste 'Green Tea' en la búsqueda
producto_seleccionado = "Black Chia Seeds"

recomendaciones = obtener_recomendaciones(producto_seleccionado, df_productos, matriz_entrenada)
print("---"*10)
print("\nRecomendaciones:")
display(recomendaciones) 


Recomendaciones para: 'Black Chia Seeds'...
------------------------------

Recomendaciones:


,product_id,ProductName,Brand,Category
1080,P001081,Dry Fruits & Nuts,Fresho Signature,Gourmet & World Food
1079,P001080,Fruits & Nuts,Fresho Signature,Gourmet & World Food
1277,P001278,Strawberry,Fresho Signature,Gourmet & World Food


In [41]:
# Prueba 2

# Elegimos un producto al azar para probar
producto_ejemplo = df_productos['ProductName'].iloc[0] # <-- cambia el producto  


# Llamamos a la función pasando el producto, la base de datos y la matriz matemática
recomendaciones = obtener_recomendaciones(producto_ejemplo, df_productos, matriz_entrenada)
print("---"*10)
print("\nRecomendaciones:")
display(recomendaciones) 


Recomendaciones para: 'Onion (Loose)'...
------------------------------

Recomendaciones:


,product_id,ProductName,Brand,Category
1,P000002,Onion,Fresho,Fruits & Vegetables
3,P000004,Potato (Loose),Fresho,Fruits & Vegetables
4,P000005,Potato,Fresho,Fruits & Vegetables


___
#### **Justificación Técnica y Evaluación**

